# 03 — Análise Multinotas: Renda (Q006) vs Desempenho por Área — ENEM 2023

Objetivo: investigar a relação entre **renda familiar (Q006)** e **desempenho** em cada uma das cinco avaliações do ENEM:
- Matemática (`NU_NOTA_MT`)
- Linguagens (`NU_NOTA_LC`)
- Ciências Humanas (`NU_NOTA_CH`)
- Ciências da Natureza (`NU_NOTA_CN`)
- Redação (`NU_NOTA_REDACAO`)

**Observações**
- Usa apenas `matplotlib` (sem seaborn).
- Não salva figuras em disco; exibe na tela.
- Requer que `../data/interim/enem_2023.parquet` exista e contenha ao menos: `Q006`, `Q006_ord` e as colunas de notas acima.
- Colunas opcionais: `Q006_rot` (rótulos legíveis), `IN_TREINEIRO`, `TP_ESCOLA_rot`.


In [ ]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

DATA_PATH = Path("../data/interim/enem_2023.parquet")
print("Lendo:", DATA_PATH)
df = pd.read_parquet(DATA_PATH)
print("Dimensões:", df.shape)

REQUIRED_NOTES = [
    "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_REDACAO"
]
REQUIRED = ["Q006", "Q006_ord"] + REQUIRED_NOTES
missing = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise RuntimeError(f"Faltam colunas necessárias: {missing}")

RENDA_ROT_COL = "Q006_rot" if "Q006_rot" in df.columns else None

if "IN_TREINEIRO" in df.columns:
    df = df[df["IN_TREINEIRO"] == 0].copy()

df = df.dropna(subset=["Q006", "Q006_ord"]).copy()
for c in REQUIRED_NOTES:
    if df[c].isna().all():
        print(f"Aviso: {c} está totalmente ausente (NaN)")
    # não dropamos geral; trataremos por coluna nas análises

df.head(3)

## 1) Correlação renda × nota (por área)
Correlação de **Pearson** entre `Q006_ord` (A=1, B=2, ...) e cada nota individual.

In [ ]:
corrs = {}
for col in REQUIRED_NOTES:
    tmp = df[["Q006_ord", col]].dropna()
    cor = tmp["Q006_ord"].corr(tmp[col]) if len(tmp) else float('nan')
    corrs[col] = cor

pd.Series(corrs, name="correlacao").sort_values(ascending=False)

## 2) Tendência da média por faixa de renda (linha)
Para cada área, calculamos a **média da nota** por `Q006_ord` e plotamos a tendência.

Se houver `Q006_rot`, mostramos também um gráfico com os **rótulos no eixo X**.

In [ ]:
for col in REQUIRED_NOTES:
    tmp = df[["Q006_ord", col]].dropna()
    if tmp.empty:
        print(f"Sem dados para {col}.")
        continue
    g = tmp.groupby("Q006_ord", as_index=False)[col].mean().sort_values("Q006_ord")

    plt.figure(figsize=(9, 4.8))
    plt.plot(g["Q006_ord"], g[col], marker="o")
    plt.title(f"Tendência da Nota Média por Faixa de Renda (Q006_ord) — {col}")
    plt.xlabel("Ordem de Renda (A=1, B=2, ...)")
    plt.ylabel("Nota Média")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

    if RENDA_ROT_COL:
        lab = df[[RENDA_ROT_COL, "Q006_ord"]].dropna().drop_duplicates().sort_values("Q006_ord")
        x = list(lab["Q006_ord"])  # ordem correta
        labs = list(lab[RENDA_ROT_COL])
        means = g.set_index("Q006_ord").reindex(x)[col]

        plt.figure(figsize=(11, 4.8))
        plt.plot(labs, means.values, marker="o")
        plt.title(f"Tendência (com rótulos) — {col}")
        plt.xlabel("Faixa de Renda (Q006_rot)")
        plt.ylabel("Nota Média")
        plt.xticks(rotation=45, ha="right")
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout()
        plt.show()

## 3) Boxplots por faixa de renda (dispersão)
Visualizamos a distribuição de notas por faixa de renda. Isso ajuda a perceber **sobreposições** e **variação intra-faixa**.

In [ ]:
for col in REQUIRED_NOTES:
    if RENDA_ROT_COL:
        order = df[[RENDA_ROT_COL, "Q006_ord"]].drop_duplicates().sort_values("Q006_ord")
        groups = [df.loc[df[RENDA_ROT_COL]==lab, col].dropna().values for lab in order[RENDA_ROT_COL]]
        labels = list(order[RENDA_ROT_COL])
    else:
        order = sorted(df["Q006"].dropna().unique())
        groups = [df.loc[df["Q006"]==lab, col].dropna().values for lab in order]
        labels = order

    plt.figure(figsize=(11.2, 5.6))
    plt.boxplot(groups, labels=labels, showfliers=False)
    plt.title(f"Boxplot da Nota por Faixa de Renda — {col}")
    plt.xlabel("Faixa de Renda")
    plt.ylabel("Nota")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

## 4) Resumo automático de observações
Gera frases simples com base nas estatísticas calculadas acima (correlações e tendências).

In [ ]:
serie_corr = pd.Series(corrs).sort_values(ascending=False)
print("Correlação renda (Q006_ord) × notas (ordem decrescente):\n")
for k, v in serie_corr.items():
    print(f"- {k}: {v:.3f}")

top = serie_corr.index[0]
print("\nObservações:")
print("- Todas as áreas tendem a apresentar relação positiva entre renda e desempenho.")
print(f"- A área com maior correlação neste recorte é: {top}.")
print("- As curvas de tendência por Q006_ord costumam ser monotônicas crescentes.")
print("- Os boxplots mostram sobreposição de distribuição entre faixas, mas a mediana sobe com a renda.")